In [1]:
import os
import numpy as np

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

d:\Thesis\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


from dotenv import dotenv_values

SECRETS = dotenv_values("../envs/graphs.env")

TRAIN_DATA_PATH = SECRETS['TRAIN_DATA_PATH']
TEST_DATA_PATH = SECRETS['TEST_DATA_PATH']
PULL_UP_STRENGTH, PULL_DOWN_STRENGTH = list(map(int, SECRETS['BETA'].split(':')))
NUM_SAMPLES = int(SECRETS['NUM_SAMPLES'])
MAX_LENGTH = int(SECRETS['MAX_LENGTH'])

In [3]:
dataset = dgl.data.CoraGraphDataset(raw_dir=TRAIN_DATA_PATH)
print(f"Number of categories: {dataset.num_classes}")

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [4]:
g = dataset[0]

In [5]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [6]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [7]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


# model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
# train(g, model)

In [8]:
# g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.074 (best 0.074), test acc: 0.090 (best 0.090)
In epoch 5, loss: 1.894, val acc: 0.416 (best 0.450), test acc: 0.435 (best 0.473)
In epoch 10, loss: 1.808, val acc: 0.368 (best 0.450), test acc: 0.403 (best 0.473)
In epoch 15, loss: 1.698, val acc: 0.556 (best 0.556), test acc: 0.560 (best 0.560)
In epoch 20, loss: 1.565, val acc: 0.652 (best 0.652), test acc: 0.666 (best 0.666)
In epoch 25, loss: 1.412, val acc: 0.652 (best 0.654), test acc: 0.695 (best 0.695)
In epoch 30, loss: 1.245, val acc: 0.692 (best 0.692), test acc: 0.718 (best 0.718)
In epoch 35, loss: 1.071, val acc: 0.722 (best 0.722), test acc: 0.734 (best 0.734)
In epoch 40, loss: 0.900, val acc: 0.730 (best 0.730), test acc: 0.754 (best 0.754)
In epoch 45, loss: 0.741, val acc: 0.742 (best 0.742), test acc: 0.764 (best 0.764)
In epoch 50, loss: 0.602, val acc: 0.750 (best 0.752), test acc: 0.769 (best 0.767)
In epoch 55, loss: 0.484, val acc: 0.762 (best 0.762), test acc: 0.776 (best 0

In [9]:
train_data = [ (np.load(TRAIN_DATA_PATH + f"/train_data/circuit{i}.npz")['adj_list'],  np.load(TRAIN_DATA_PATH + f"/train_data/circuit{i}.npz")['feature_matrix']) for i in range(1, 1000+1) ]

In [10]:
train_data[0][0].shape, train_data[0][1].shape

((2, 15), (16, 5))

In [19]:
g = dgl.graph((train_data[0][0][0], train_data[0][0][1]))
g.ndata['x'] = torch.tensor(train_data[0][1], dtype=torch.float32)

In [20]:
g

Graph(num_nodes=16, num_edges=15,
      ndata_schemes={'x': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={})

In [16]:
train_data[0][0][1]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [24]:
g.ndata['x']

tensor([[ 0.,  0., 22., 43., 22.],
        [ 1.,  4., 22., 43.,  1.],
        [ 2.,  4., 22., 43.,  2.],
        [ 3.,  0., 22., 43., 47.],
        [ 4.,  3., 22., 43., 36.],
        [ 5.,  2., 22., 43.,  4.],
        [ 6.,  2., 22., 43., 27.],
        [ 7.,  1., 22., 43., 30.],
        [ 8.,  3., 22., 43., 46.],
        [ 9.,  0., 22., 43., 14.],
        [10.,  4., 22., 43., 19.],
        [11.,  1., 22., 43., 16.],
        [12.,  4., 22., 43., 29.],
        [13.,  0., 22., 43., 30.],
        [14.,  1., 22., 43., 20.],
        [15.,  1., 22., 43., 20.]])